<a href="https://colab.research.google.com/github/anujkadu/InfinityPool-Work/blob/main/Infinity_Pool_TensorFlow_Manual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate Dataset (will default a loan)

In [70]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Generate synthetic financial dataset
np.random.seed(42)
N = 1000
df = pd.DataFrame({
    'age': np.random.randint(21, 60, N),
    'income': np.random.randint(20000, 150000, N),
    'credit_score': np.random.randint(300, 900, N),
    'loan_amount': np.random.randint(50000, 1000000, N),
    'loan_term': np.random.choice([12, 24, 36, 48, 60], N),
    'past_defaults': np.random.poisson(0.5, N)
})
df['will_default'] = (
    (df['credit_score'] < 600).astype(int) |
    ((df['loan_amount'] / df['income']) > 10).astype(int) |
    (df['past_defaults'] > 2).astype(int)
)

# Convert to tensorflow compatible format

In [71]:
X_raw = df.drop("will_default", axis=1).values
y = df["will_default"].values
scaler = MinMaxScaler()
X = scaler.fit_transform(X_raw)

X_tensor = tf.constant(X, dtype=tf.float32)
y_tensor = tf.constant(y.reshape(-1, 1), dtype=tf.float32)

# Manually declare weights for each neuron of our model

In [72]:
W1 = tf.Variable([
    [0.05, 0.1, 0.2, 0.0, 0.1, 0.3, 0.0, 0.15],
    [0.0, 0.0, 0.1, 0.2, 0.0, 0.2, 0.1, 0.1],
    [0.2, 0.2, 0.0, 0.1, 0.3, 0.0, 0.1, 0.0],
    [0.1, 0.0, 0.0, 0.3, 0.1, 0.0, 0.2, 0.05],
    [0.05, 0.1, 0.1, 0.1, 0.05, 0.1, 0.0, 0.0],
    [0.3, 0.2, 0.1, 0.0, 0.0, 0.1, 0.3, 0.2]
], dtype=tf.float32)
b1 = tf.Variable([0.1, 0.2, 0.0, -0.1, 0.1, 0.0, 0.05, -0.05], dtype=tf.float32)

# Second hidden layer (8 → 4)
W2 = tf.Variable([
    [0.10, 0.20, 0.00, 0.10],
    [0.00, 0.10, 0.20, 0.00],
    [0.20, 0.10, 0.10, 0.00],
    [0.10, 0.00, 0.00, 0.30],
    [0.10, 0.00, 0.30, 0.10],
    [0.00, 0.20, 0.10, 0.00],
    [0.20, 0.10, 0.00, 0.10],
    [0.00, 0.00, 0.10, 0.20]
], dtype=tf.float32)
b2 = tf.Variable([0.00, 0.10, -0.05, 0.20], dtype=tf.float32)

# Output layer (4 → 1)
W3 = tf.Variable([[0.2], [0.1], [-0.1], [0.3]], dtype=tf.float32)
b3 = tf.Variable([0.1], dtype=tf.float32)

# Calculate linear transformation and use activation function

In [73]:
Z1 = tf.matmul(X_tensor, W1) + b1
A1 = tf.nn.relu(Z1)

Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)

Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)  # final output: probability

print("Predicted Probabilities (first 5):")
print(A3.numpy()[:5])


Predicted Probabilities (first 5):
[[0.57750833]
 [0.571759  ]
 [0.5744479 ]
 [0.5485885 ]
 [0.5649627 ]]


# Declare optimizer and loss function

In [85]:

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.BinaryCrossentropy()


# epsilon = 1e-7  # for numerical stability to prevent log(0)
# loss_fn = tf.reduce_mean(
#     -y_tensor * tf.math.log(A3 + epsilon) - (1 - y_tensor) * tf.math.log(1 - A3 + epsilon)
# )


# Calculate Loss and Accuracy before training

In [86]:
loss = loss_fn(y_tensor, A3)
y_pred_before = tf.cast(A3 > 0.5, dtype=tf.int32)
y_true = tf.cast(y_tensor, dtype=tf.int32)
accuracy_before = tf.reduce_mean(tf.cast(tf.equal(y_pred_before, y_true), tf.float32))

print(f"Loss: {loss.numpy():.4f}")
#print("Loss = ",loss_fn)
print(f"Accuracy Before Update: {accuracy_before.numpy() * 100:.4f}%")


Loss: 0.6682
Accuracy Before Update: 63.8000%


# Sample prediction before training

In [87]:
new_customer = pd.DataFrame([{
    'age': 29,
    'income': 25000,
    'credit_score': 490,
    'loan_amount': 450000,
    'loan_term': 60,
    'past_defaults': 3
}])

# Normalize
new_customer_scaled = scaler.transform(new_customer)
X_new = tf.constant(new_customer_scaled, dtype=tf.float32)

# Manual forward pass BEFORE training
Z1 = tf.matmul(X_new, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)

print("BEFORE Training → Probability of default:", A3.numpy()[0][0])


BEFORE Training → Probability of default: 0.5693356


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(


# Calculate Gradients

tf.matmul: matrix multiplication

tf.nn.relu: zeroes out negatives (adds non-linearity)

tf.nn.sigmoid: converts raw score into probability between 0 and 1

In [88]:
with tf.GradientTape() as tape:
    Z1 = tf.matmul(X_tensor, W1) + b1
    A1 = tf.nn.relu(Z1)
    Z2 = tf.matmul(A1, W2) + b2
    A2 = tf.nn.relu(Z2)
    Z3 = tf.matmul(A2, W3) + b3
    A3 = tf.nn.sigmoid(Z3)
    loss = loss_fn(y_tensor, A3)

# Compute gradients
grads = tape.gradient(loss, [W1, b1, W2, b2, W3, b3])
print("Gradients calculated for all weights and biases.")


Gradients calculated for all weights and biases.


# 1 Sample Epoch

In [90]:
# Apply gradients to update weights
optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2, W3, b3]))

# Rerun forward pass to see new predictions
Z1 = tf.matmul(X_tensor, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)

# Accuracy after update
y_pred = tf.cast(A3 > 0.5, dtype=tf.int32)
y_true = tf.cast(y_tensor, dtype=tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))

print("Predicted Probabilities After Update (first 5):")
print(A3.numpy()[:5])
print(f"Accuracy After Update: {accuracy.numpy() * 100:.4f}%")

# Print updated weights
print("\nUpdated Weights:")
print("W1:\n", W1.numpy())
print("W2:\n", W2.numpy())
print("W3:\n", W3.numpy())


Predicted Probabilities After Update (first 5):
[[0.6092458 ]
 [0.59981   ]
 [0.6042871 ]
 [0.56509566]
 [0.59017265]]
Accuracy After Update: 63.8000%

Updated Weights:
W1:
 [[ 0.06997666  0.08016128  0.21995932  0.01998376  0.11991894  0.31983873
   0.01997955  0.16996722]
 [-0.01859707  0.0131257   0.0823295   0.18050514 -0.01583617  0.1868743
   0.08123872  0.09125167]
 [ 0.18001519  0.21989517 -0.01997356  0.08000957  0.28005266 -0.01989518
   0.08001331 -0.01997813]
 [ 0.11998644 -0.01990647  0.01997641  0.3199913   0.11995304  0.01990647
   0.21998812  0.06998056]
 [ 0.06997854  0.08014826  0.1199626   0.11998606  0.0699255   0.11985174
   0.0199812   0.01996941]
 [ 0.3199202   0.18054435  0.11986087  0.01994788  0.01972388  0.11945565
   0.3199301   0.21988851]]
W2:
 [[ 0.1199696   0.21993947 -0.01993947  0.11997966]
 [ 0.01997864  0.1199575   0.1800425   0.0199857 ]
 [ 0.21997571  0.11995166  0.08004835  0.01998375]
 [ 0.1199688   0.01993786 -0.01993786  0.31997913]
 [ 0.119894

# Running multiple epochs

In [91]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.BinaryCrossentropy()

# Training loop
epochs = 100

for epoch in range(1, epochs + 1):
    with tf.GradientTape() as tape:
        # Forward pass
        Z1 = tf.matmul(X_tensor, W1) + b1
        A1 = tf.nn.relu(Z1)
        Z2 = tf.matmul(A1, W2) + b2
        A2 = tf.nn.relu(Z2)
        Z3 = tf.matmul(A2, W3) + b3
        A3 = tf.nn.sigmoid(Z3)

        # Compute loss
        loss = loss_fn(y_tensor, A3)

    # Compute gradients
    grads = tape.gradient(loss, [W1, b1, W2, b2, W3, b3])
    # Update weights
    optimizer.apply_gradients(zip(grads, [W1, b1, W2, b2, W3, b3]))

    # Compute accuracy
    y_pred = tf.cast(A3 > 0.5, dtype=tf.int32)
    y_true = tf.cast(y_tensor, dtype=tf.int32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_pred, y_true), tf.float32))

    # Print progress every 10 epochs
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} → Loss: {loss.numpy():.4f} | Accuracy: {accuracy.numpy() * 100:.2f}%")
    # print("\nUpdated Weights:")
    # print("W1:\n", W1.numpy())
    # print("W2:\n", W2.numpy())
    # print("W3:\n", W3.numpy())

Epoch 001 → Loss: 0.6624 | Accuracy: 63.80%
Epoch 010 → Loss: 0.6448 | Accuracy: 63.80%
Epoch 020 → Loss: 0.6025 | Accuracy: 63.80%
Epoch 030 → Loss: 0.5138 | Accuracy: 75.10%
Epoch 040 → Loss: 0.4042 | Accuracy: 82.40%
Epoch 050 → Loss: 0.3186 | Accuracy: 85.90%
Epoch 060 → Loss: 0.2903 | Accuracy: 86.20%
Epoch 070 → Loss: 0.2871 | Accuracy: 86.60%
Epoch 080 → Loss: 0.2872 | Accuracy: 86.00%
Epoch 090 → Loss: 0.2863 | Accuracy: 86.60%
Epoch 100 → Loss: 0.2858 | Accuracy: 86.10%


# Prediction based on fine tuned model

In [92]:
new_customer = pd.DataFrame([{
    'age': 29,
    'income': 25000,
    'credit_score': 490,
    'loan_amount': 450000,
    'loan_term': 60,
    'past_defaults': 3
}])

# Normalize
new_customer_scaled = scaler.transform(new_customer)
X_new = tf.constant(new_customer_scaled, dtype=tf.float32)

# Manual forward pass BEFORE training
Z1 = tf.matmul(X_new, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
Z3 = tf.matmul(A2, W3) + b3
A3 = tf.nn.sigmoid(Z3)

print("BEFORE Training → Probability of default:", A3.numpy()[0][0])


BEFORE Training → Probability of default: 0.9992362


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
